In [1]:
# Necessary imports
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Layout, widgets, Output, RadioButtons, Checkbox, Dropdown
from IPython.display import display
from scipy.special import erfc
from scipy.signal import upfirdn

# Function to calculate ASK modulation errors
def ask_errors_sin(k, M, nsamp, EbN0_db, matched_filter_type='Normal'):
    L = 2**k
    SNR_db = EbN0_db - 10 * np.log10(nsamp / (2 * k))
    SNR_linear = 10 ** (SNR_db / 10)
    x = 2 * np.floor(L * np.random.rand(M)) - L + 1

    h = np.cos(2 * np.pi * np.arange(1, nsamp + 1) / nsamp)
    h = h / np.sqrt(np.sum(h**2))

    y = upfirdn(h, x, up=nsamp)
    y = y[:M * nsamp]

    P_x = np.mean(y ** 2)
    noise_variance = P_x / SNR_linear
    noise = np.random.normal(0, np.sqrt(noise_variance), len(y))
    y_noisy = y + noise

    matched = h[::-1] if matched_filter_type == 'Reversed' else h
    yrx = np.convolve(y_noisy, matched, mode='full')
    z = yrx[nsamp - 1:M * nsamp:nsamp]

    levels = np.arange(-L + 1, L, 2)
    z_decided = levels[np.abs(levels[:, None] - z).argmin(axis=0)]

    errors = np.count_nonzero(x != z_decided)

    return errors

# Initialize parameters
M = 10000
EbN0_db = np.arange(0, 21, 2)

checkbox_4qam4 = Checkbox(value=True, description='4-ASK')
checkbox_8qam4 = Checkbox(value=False, description='8-ASK')
checkbox_16qam4 = Checkbox(value=False, description='16-ASK')
nsamp_dropdown4 = Dropdown(options=[4, 8, 16, 32, 64], value=16, description='Samples per Symbol:')
matched_radio = RadioButtons(options=['Normal', 'Reversed'], value='Normal', description='Matched Filter:')
plot_output4 = Output()

# Function to plot the selected modulations
def plot_selected_modulations4(change):
    with plot_output4:
        plot_output4.clear_output(wait=True)
        nsamp = nsamp_dropdown4.value
        matched_filter_type = matched_radio.value
        plt.figure(figsize=(10, 7))
        colors = {'4-ASK': 'red', '8-ASK': 'green', '16-ASK': 'blue'}

        for k, checkbox, color in zip([2, 3, 4], [checkbox_4qam4, checkbox_8qam4, checkbox_16qam4], colors.values()):
            if checkbox.value:
                L = 2**k
                modulation_name = f'{L}-ASK'
                ber = np.zeros(len(EbN0_db))
                for index, eb_n0 in enumerate(EbN0_db):
                    ber[index] = ask_errors_sin(k, M, nsamp, eb_n0, matched_filter_type) / (M * np.log2(L))

                plt.semilogy(EbN0_db, ber, 'o', label=f'Experimental {modulation_name}', color=color)
                ber_theoretical = (((L-1)/L) * erfc(np.sqrt(10**(EbN0_db / 10) * (3 * np.log2(L)) / (L**2 - 1)))) / k
                plt.semilogy(EbN0_db, ber_theoretical, linestyle='-', label=f'Theoretical {modulation_name}', color=color)

        plt.grid(True, which='both')
        plt.xlabel("Eb/N0 (dB)")
        plt.ylabel("BER")
        plt.title(f'Theoretical and Experimental BER of ASK Modulations [nsamp={nsamp}]')
        plt.legend()
        plt.show()

# Attach the plot function to UI components
checkbox_4qam4.observe(plot_selected_modulations4, names='value')
checkbox_8qam4.observe(plot_selected_modulations4, names='value')
checkbox_16qam4.observe(plot_selected_modulations4, names='value')
nsamp_dropdown4.observe(plot_selected_modulations4, names='value')
matched_radio.observe(plot_selected_modulations4, names='value')

# Set up the UI layout
inputs1 = widgets.HBox([nsamp_dropdown4])
inputs2 = widgets.HBox([matched_radio], layout=Layout(margin="0 0 0 20px"))
inputs12 = widgets.HBox([inputs1, inputs2], layout=Layout(align_items='center'))
inputs3 = widgets.VBox([checkbox_4qam4, checkbox_8qam4, checkbox_16qam4])

inputs = widgets.VBox([inputs12, inputs3])

ui = widgets.HBox([inputs], layout=Layout(align_items='center'))

# Display the UI and plot
display(ui, plot_output4)

# Initial plot call
plot_selected_modulations4(None)


Output()